In [1]:
import netCDF4 as nc
import pandas as pd
import math 
import numpy as np
import os


In [17]:
district_coordinates_df = pd.read_csv("D:\sop\district_bounding_boxes.csv")
nc_parent_folder_path = "D:/sop/Bika/2"
list_of_files = os.listdir(nc_parent_folder_path)
#Choosing weather forecast made at 12 PM 
list_of_chosen_files = sorted([x for x in list_of_files if "12.f024.grib2.nc" in x])

In [18]:
def return_mid_point(min_coord, max_coord, is_lat):
    lower_bound = 0
    upper_bound = 0
    step = 0.25

    if(is_lat==True):
        # lower_bound = 24
        # upper_bound = 50
        lower_bound=21
        upper_bound=31
    else:
        lower_bound = 69#234
        upper_bound = 80#295
    
    series = np.arange(lower_bound, upper_bound, step)
    filtered_numbers = series[(series >= min_coord) & (series <= max_coord)]
    if len(filtered_numbers) == 0:  
       return (min_coord + max_coord) / 2
    chosen_coord = filtered_numbers[math.floor(len(filtered_numbers)/2)]
    return chosen_coord

districts_list = []
lats_list = []
longs_list = []

In [19]:
#Extracting list of districts, latitude and longitude mapped to given district at 0.25 resolution
for index, row in district_coordinates_df.iterrows():
    district_name = row["District"]
    min_lon = row["Min_Lon"]
    min_lat = row["Min_Lat"]
    max_lon = row["Max_Lon"]
    max_lat = row["Max_Lat"]

    lat =  float(return_mid_point(min_lat,max_lat,True))
    lon =  float(return_mid_point(min_lon,max_lon, False))
    districts_list.append(district_name)
    lats_list.append(lat)
    longs_list.append(lon)

latitudes = []
longitudes = []
precip_rates = []
rel_humidity = []
cloud_water = []
dates = []
districts = []
temperatures = []
pressures = []
snow_depths = []
dew_temps = []
spec_humidity = []
total_cloud_covers = []
total_precipitation = []
max_temperatures = []
min_temperatures = []
wind_speeds = []
visibility = []
volumetric_soil_moisture = []
vegetation = []
soil_type = []

In [20]:
def data_prep(file_name):
    """Maps temperature to latitude, longitude and saves to arrays"""
    date = file_name.split(".")[2][:-2]
    file_path = nc_parent_folder_path + ("/") + (file_name)

    #Opening the .nc file
    dataset = nc.Dataset(file_path)
    lat = dataset.variables['lat'][:]
    lon = dataset.variables['lon'][:]
    # print(dataset.variables.keys())
    # tmp_l1 = dataset.variables['TMP_L1'][:]
    # pres_l1 = dataset.variables['PRES_L1'][:]
    # cloud_wat = dataset.variables['C_WAT_L200'][:]
    # precip_rat = dataset.variables['PRATE_L1'][:]
    rel_hum = dataset.variables['R_H_L103'][:]
    # sno_dep = dataset.variables["SNO_D_L1"][:]
    dew_points = dataset.variables["DPT_L103"][:]
    spec_hum = dataset.variables["SPF_H_L103"][:]
    # tot_cloud_cov = dataset.variables["T_CDC_L244"][:]
    # tot_preicp = dataset.variables["A_PCP_L1_Accum_1"][:]
    max_temps = dataset.variables['T_MAX_L103_Max_1'][:]
    min_temps = dataset.variables['T_MIN_L103_Min_1'][:]
    # wind_sp = dataset.variables['GUST_L1'][:]
    # vis = dataset.variables['VIS_L1'][:]
    # vol_soil = dataset.variables['SOILW_Y106'][:]
    # veg = dataset.variables['VEG_L1'][:]
    # sty = dataset.variables['SOTYP_L1'][:]


    for i in range(len(districts_list)):
            latitudes.append(lats_list[i])
            longitudes.append(longs_list[i])
            districts.append(districts_list[i])
            dates.append(date)
            lat_index = np.where(lat == lats_list[i])[0][0]
            long_index = np.where(lon == longs_list[i])[0][0]
            # temperatures.append(tmp_l1[0][lat_index][long_index])
            # pressures.append(pres_l1[0][lat_index][long_index])
            # cloud_water.append(cloud_wat[0][lat_index][long_index])
            # precip_rates.append(precip_rat[0][lat_index][long_index])
            rel_humidity.append(rel_hum[0][lat_index][long_index])
            # snow_depths.append(sno_dep[0][lat_index][long_index])
            dew_temps.append(dew_points[0][lat_index][long_index])
            spec_humidity.append(spec_hum[0][lat_index][long_index])
            # total_cloud_covers.append(tot_cloud_cov[0][lat_index][long_index])
            # total_precipitation.append(tot_preicp[0][lat_index][long_index])
            max_temperatures.append(max_temps[0][lat_index][long_index])
            min_temperatures.append(min_temps[0][lat_index][long_index])
            # visibility.append(vis[0][lat_index][long_index])
            # wind_speeds.append(wind_sp[0][lat_index][long_index])
            # volumetric_soil_moisture.append(vol_soil[0][lat_index][long_index])
            # vegetation.append(veg[0][lat_index][long_index])
            # soil_type.append(sty[0][lat_index][long_index])

In [21]:
for filename in list_of_chosen_files:
    data_prep(filename)

In [22]:
df = pd.DataFrame(
    {
        'Date': dates,
        'District': districts,
        'Latitude': latitudes,
        'Longitude': longitudes,
        # 'Volumetric Soil Moisture Content': volumetric_soil_moisture
        # 'Visibility': visibility,
        # 'Wind Speed': wind_speeds,
        'Maximum Temperature': max_temperatures,
        'Minimum Temperature': min_temperatures,
        # 'Total Cloud Cover': total_cloud_covers,
        # 'Total Precipitation': total_precipitation,
        # 'Snow Depth': snow_depths,
        'Dewpoint Temperatures': dew_temps,
        'Specific Humidity': spec_humidity,
        # 'Temperature': temperatures,
        # 'Pressure':pressures,
        # 'Cloud Water': cloud_water,
        # 'Precipitation Rate': precip_rates,
        'Relative Humidity': rel_humidity
        # 'Vegetation':vegetation,
        # 'Soil Type':soil_type
    }
)

In [23]:
df.to_csv("D:/sop/Final_csv/bika2.csv", index=False)